### Datasets
http://www.manythings.org/anki/  

In [3]:
import pandas as pd
import numpy as np
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

Using TensorFlow backend.


In [1]:
# Run this cell to mount your Google Drive.
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
lines= pd.read_table('/content/drive/My Drive/MT/hin.txt', names=['eng', 'hin'])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  """Entry point for launching an IPython kernel.


In [45]:
lines.shape

(2831, 2)

In [0]:
# Lowercase all characters
lines.eng=lines.eng.apply(lambda x: x.lower())
lines.hin=lines.hin.apply(lambda x: x.lower())

In [0]:
# Remove quotes
lines.eng=lines.eng.apply(lambda x: re.sub("'", '', x))
lines.hin=lines.hin.apply(lambda x: re.sub("'", '', x))

In [0]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines.eng=lines.eng.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines.hin=lines.hin.apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [0]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines.eng=lines.eng.apply(lambda x: x.translate(remove_digits))
lines.hin = lines.hin.apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

In [0]:
# Remove extra spaces
lines.eng=lines.eng.apply(lambda x: x.strip())
lines.hin=lines.hin.apply(lambda x: x.strip())
lines.eng=lines.eng.apply(lambda x: re.sub(" +", " ", x))
lines.hin=lines.hin.apply(lambda x: re.sub(" +", " ", x))

In [0]:
# Add start and end tokens to target sequences
lines.hin = lines.hin.apply(lambda x : 'START_ '+ x + ' _END')

In [52]:
lines.sample(10)

,eng,hin
84,i like both,START_ मुझे दोनो पसंद हैं। _END
283,were in a hurry,START_ हम जल्दी में हैं। _END
1791,you must get rid of such a habit,START_ तुम्हे इस तरह की आदत तोड़ देनी चाहिए। _END
1148,i dont know what to study,START_ मुझे समझ में नहीं आ रहा कि क्या पढ़ूँ। ...
2791,this morning the weather was so bad that i had...,START_ आज सुबह मौसम इतना खराब था कि मुझे टैक्स...
2733,this car has enough power to go up the mountai...,START_ गाड़ी में आसानी से पहाड़ चढ़ने लायक जान...
1084,the haze enveloped london,START_ लंदन में धुंध छा गई थी। _END
406,its mine not his,START_ मेरा है उसका नहीं। _END
2244,all you have to do is follow his advice,START_ तुम्हे बस उसकी सलाह के मुताबिक काम करना...
538,i agree to this plan,START_ मैं इस योजना से सहमत हूँ। _END


In [0]:
# Vocabulary of English
all_eng_words=set()
for eng in lines.eng:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

# Vocabulary of Hindi
all_hindi_words=set()
for hin in lines.hin:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [54]:
# Max Length of source sequence
length_list=[]
for l in lines.eng:
    length_list.append(len(l.split(' ')))
max_length_src = np.max(length_list)
max_length_src

22

In [55]:
# Max Length of target sequence
length_list=[]
for l in lines.hin:
    length_list.append(len(l.split(' ')))
max_length_tar = np.max(length_list)
max_length_tar

27

In [56]:
input_words = sorted(list(all_eng_words))
target_words = sorted(list(all_hindi_words))
num_encoder_tokens = len(all_eng_words)
num_decoder_tokens = len(all_hindi_words)
num_encoder_tokens, num_decoder_tokens

(2361, 3003)

In [57]:
num_decoder_tokens += 1 # For zero padding
num_decoder_tokens

3004

In [0]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [0]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [60]:
lines = shuffle(lines)
lines.head(10)

,eng,hin
2325,if only i had known the answer yesterday,START_ काश मुझे यह जवाब कल पता होता _END
814,he is used to hard work,START_ उसको मेहनत करने की आदत है। _END
258,he speaks arabic,START_ वह अरबी बोलता है। _END
858,they stood face to face,START_ वे आमनेसामने खड़े हुए। _END
2255,how about adding a little bit more salt,START_ थोड़ा और नमक डाल दोगे _END
2770,he promised me that he would be more careful i...,START_ उसने मुझसे वादा किया था कि वह आगे से ज़...
994,did the police arrest tom,START_ पुलीस ने टॉम को गिरफ़्तार किया क्या _END
1740,it wont take long to do the job,START_ इस काम को करने में ज़्यादा देर नहीं लगे...
821,he was more than a king,START_ वह एक राजा से बहुत ज़्यादा था। _END
1300,the room smelled of tobacco,START_ कमरे में तम्बाकू की महक थी। _END


In [61]:
# Train - Test Split
X, y = lines.eng, lines.hin
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.05)
X_train.shape, X_test.shape

((2689,), (142,))

#### Save the train and test dataframes for reproducing the results later, as they are shuffled.

In [0]:
"""X_train.to_pickle('Weights_Hin/X_train.pkl')
X_test.to_pickle('Weights_Hin/X_test.pkl')"""

In [0]:
def generate_batch(X = X_train, y = y_train, batch_size = 32):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder - Decoder Model Architecture

In [0]:
latent_dim = 50

In [0]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [0]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [0]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [0]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 32

epochs = 100

In [70]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)

Epoch 1/100
84/84 [==============================] - 14s 169ms/step - loss: 7.0257 - acc: 0.1242 - val_loss: 6.0294 - val_acc: 0.1295
Epoch 2/100
84/84 [==============================] - 12s 137ms/step - loss: 5.9747 - acc: 0.1281 - val_loss: 5.9494 - val_acc: 0.1320
Epoch 3/100
84/84 [==============================] - 11s 136ms/step - loss: 5.8787 - acc: 0.1282 - val_loss: 5.8240 - val_acc: 0.1352
Epoch 4/100
84/84 [==============================] - 12s 137ms/step - loss: 5.7413 - acc: 0.1283 - val_loss: 5.7252 - val_acc: 0.1342
Epoch 5/100
84/84 [==============================] - 11s 136ms/step - loss: 5.5893 - acc: 0.1302 - val_loss: 5.6677 - val_acc: 0.1395
Epoch 6/100
84/84 [==============================] - 11s 136ms/step - loss: 5.4801 - acc: 0.1442 - val_loss: 5.6191 - val_acc: 0.1487
Epoch 7/100
84/84 [==============================] - 11s 137ms/step - loss: 5.3842 - acc: 0.1530 - val_loss: 5.4654 - val_acc: 0.1719
Epoch 8/100
84/84 [==============================] - 12s 137ms

### Always remember to save the weights

In [0]:
model.save_weights('nmt_weights.h5')

### Load the weights, if you close the application

In [0]:
model.load_weights('nmt_weights.h5')

### Inference Setup

In [0]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)

### Decode sample sequeces

In [0]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

### Evaluation on Train Dataset

In [0]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1

In [76]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: is that a cat
Actual Hindi Translation:  वह बिल्ली है क्या 
Predicted Hindi Translation:  वह खबर रखना चाहिए। 


In [77]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: she said to herself i am very happy
Actual Hindi Translation:  उसने अपने आप से कहा मैं बहुत खुश हूँ 
Predicted Hindi Translation:  उसने अपने पैसे करने की कोशिश करी थी। 


In [78]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: a fire broke out near my house
Actual Hindi Translation:  मेरे घर के पड़ोस में आग लग गई थी। 
Predicted Hindi Translation:  मेरे घर के लिए बहुत सारे विद्यार्थी थे। 


In [79]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_train[k:k+1].values[0])
print('Actual Hindi Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: im very thirsty
Actual Hindi Translation:  मुझे बहुत प्यास लगी है। 
Predicted Hindi Translation:  मुझे बहुत प्यास लगी है। 


### Evaluation on Validation Dataset

In [0]:
val_gen = generate_batch(X_test, y_test, batch_size = 1)
k=-1

In [82]:
k+=1
(input_seq, actual_output), _ = next(val_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input English sentence:', X_test[k:k+1].values[0])
print('Actual Hindi Translation:', y_test[k:k+1].values[0][6:-4])
print('Predicted Hindi Translation:', decoded_sentence[:-4])

Input English sentence: i met mary yesterday
Actual Hindi Translation:  मैं कल मेरी से मिला था। 
Predicted Hindi Translation:  मैं तुम्हारे पास एक मिलने का जवाब है। 
